In [ ]:
Project Tomato Leaf Classification

# Import the libraries

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import cv2
from pathlib import Path
from skimage.io import imread, imsave
from skimage.transform import resize
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras import layers as L
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import RMSprop, Adam
from keras.applications import vgg16

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Loading the data

!unzip '/content/drive/MyDrive/Adv. Ai by Sundaram- 9th Sep 2023/14th Oct 2023/tomato_leaf_images.zip'

Streaming output truncated to the last 5000 lines.
  inflating: tomato_leaf_images/train/MagnesiumDeficiency/IMG_20190407_111224_256.jpg  
  inflating: __MACOSX/tomato_leaf_images/train/MagnesiumDeficiency/._IMG_20190407_111224_256.jpg  
  inflating: tomato_leaf_images/train/MagnesiumDeficiency/IMG_20190407_115252_BURST001_COVER_256.jpg  
  inflating: __MACOSX/tomato_leaf_images/train/MagnesiumDeficiency/._IMG_20190407_115252_BURST001_COVER_256.jpg  
  inflating: tomato_leaf_images/train/MagnesiumDeficiency/IMG_20190407_105309_256.jpg  
  inflating: __MACOSX/tomato_leaf_images/train/MagnesiumDeficiency/._IMG_20190407_105309_256.jpg  
  inflating: tomato_leaf_images/train/MagnesiumDeficiency/IMG_20190407_102538_Bokeh_256.jpg  
  inflating: __MACOSX/tomato_leaf_images/train/MagnesiumDeficiency/._IMG_20190407_102538_Bokeh_256.jpg  
  inflating: tomato_leaf_images/train/MagnesiumDeficiency/IMG_20190407_115828_256.jpg  
  inflating: __MACOSX/tomato_leaf_images/train/MagnesiumDeficiency/._IM

In [ ]:
# Prepare the dataset for training and validation

training_data = Path('/content/tomato_leaf_images/train')
validation_data = Path('/content/tomato_leaf_images/val')

In [ ]:
print(training_data, validation_data)

/content/tomato_leaf_images/train /content/tomato_leaf_images/val


In [ ]:
# Labels the files name by using integer
labels_dict = {'AmericanLeafMiner':0, 'Healthy':1, 'MagnesiumDeficiency':2,
               'SerpentineLeafMiner':3}

In [ ]:
print(labels_dict)

{'AmericanLeafMiner': 0, 'Healthy': 1, 'MagnesiumDeficiency': 2, 'SerpentineLeafMiner': 3}


In [ ]:
# creating a dataframe for the training dataset

train_df = []

for folder in os.listdir(training_data):
  imgs_path = training_data / folder
#   print(imgs_path)

  # get the list of all the images stored in that directory
  imgs = sorted(imgs_path.glob('*.jpg'))
#   print(imgs)

  # store each image path and corresponding lable
  for img_name in imgs:
    train_df.append((str(img_name), labels_dict[folder]))
# print(train_df)

# creating dataframe
train_df = pd.DataFrame(train_df, columns=['image', 'label'], index=None)
# print(train_df)

# shuffle the dataset
train_df = train_df.sample(frac=1.).reset_index(drop=True)
# print(train_df)


In [ ]:
train_df

,image,label
0,/content/tomato_leaf_images/train/MagnesiumDef...,2
1,/content/tomato_leaf_images/train/SerpentineLe...,3
2,/content/tomato_leaf_images/train/AmericanLeaf...,0
3,/content/tomato_leaf_images/train/Healthy/resi...,1
4,/content/tomato_leaf_images/train/Healthy/IMG_...,1
...,...,...
4249,/content/tomato_leaf_images/train/AmericanLeaf...,0
4250,/content/tomato_leaf_images/train/MagnesiumDef...,2
4251,/content/tomato_leaf_images/train/AmericanLeaf...,0
4252,/content/tomato_leaf_images/train/MagnesiumDef...,2


In [ ]:
# creating a dataframe for the training dataset

valid_df = []

for folder in os.listdir(validation_data):
  imgs_path = validation_data / folder

  # get the list of all the images stored in that directory
  imgs = sorted(imgs_path.glob('*.jpg'))

  # store each image path and corresponding lable
  for img_name in imgs:
    valid_df.append((str(img_name), labels_dict[folder]))

# creating dataframe
valid_df = pd.DataFrame(valid_df, columns=['image', 'label'], index=None)

# shuffle the dataset
valid_df = valid_df.sample(frac=1.).reset_index(drop=True)


In [ ]:
valid_df

,image,label
0,/content/tomato_leaf_images/val/AmericanLeafMi...,0
1,/content/tomato_leaf_images/val/AmericanLeafMi...,0
2,/content/tomato_leaf_images/val/SerpentineLeaf...,3
3,/content/tomato_leaf_images/val/MagnesiumDefic...,2
4,/content/tomato_leaf_images/val/MagnesiumDefic...,2
...,...,...
466,/content/tomato_leaf_images/val/AmericanLeafMi...,0
467,/content/tomato_leaf_images/val/SerpentineLeaf...,3
468,/content/tomato_leaf_images/val/SerpentineLeaf...,3
469,/content/tomato_leaf_images/val/MagnesiumDefic...,2


In [ ]:
# Configuration

# dimensions to consider for the image
img_rows, img_cols, img_channels = 224,224,3

# batch_size
batch_size = 8
# total no of classes
nb_classes = 4

In [ ]:
# Data Augmentation

import imgaug as ia
from imgaug import augmenters as iaa
seed = 1234
ia.seed(seed)

# Augmentation sequence

seq = iaa.OneOf([
    iaa.Fliplr(),
    iaa.Affine(rotate=20),
    iaa.Multiply((1.2, 1.5))])

In [ ]:
# Data Generator

def data_generator(data, batch_size, preprocessing_fn = None, is_validation_data=False):
  n = len(data)
  nb_batches = int(np.ceil(n/batch_size))
  indices = np.arange(n)

  while True:
    if not is_validation_data:
      np.random.shuffle(indices)

    for i in range(nb_batches):
      next_batch_indices = indices[i*batch_size:(i+1)*batch_size]
      nb_examples = len(next_batch_indices)

      # Define two numpy array for containing batch data and labels

      batch_data = np.zeros((nb_examples, img_rows, img_cols, img_channels), dtype =np.float32)
      batch_labels = np.zeros((nb_examples, nb_classes), dtype = np.float32)

      # Process the next batch

      for j, idx in enumerate(next_batch_indices):
        img = cv2.imread(data.iloc[idx]['image'])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        label = data.iloc[idx]['label']

        if not is_validation_data:
          img = seq.augment_image(img)

        img = cv2.resize(img, (img_rows, img_cols)).astype(np.float32)
        batch_data[j] = img
        batch_labels[j] = to_categorical(label, num_classes=nb_classes)

        if preprocessing_fn is not None:
          batch_data = preprocessing_fn(batch_data)

        yield batch_data, batch_labels

In [ ]:
# VGG16 model

preprocessing_fn = vgg16.preprocess_input

train_data_gen = data_generator(train_df, batch_size, preprocessing_fn)
valid_data_gen = data_generator(valid_df, batch_size, preprocessing_fn, is_validation_data=True)

# Transfer Learning with Fine Tuning

In [ ]:
def get_base_model():
  base_model = vgg16.VGG16(input_shape=(img_rows, img_cols, img_channels), weights="imagenet",
                           include_top = True)
  return base_model

In [ ]:
# get the base model
base_model = get_base_model()

# get the output of the second last dense layer
base_model_output = base_model.layers[-2].output

# add new layers
x = L.Dropout(0.5, name='drop2')(base_model_output)
output = L.Dense(nb_classes, activation='softmax', name='fc3')(x)

# Define a new model

model = Model(base_model.input, output)

553467096/553467096 [==============================] - 2s 0us/step


In [ ]:
# Freeze all the base model layers
"""
for layer in base_model.layers[:-1]:
  layer.trainable = False
"""

In [ ]:
# Compile the model and check it

optimizer = RMSprop(0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
# Model Traning

# Use early stopping
es = EarlyStopping(patience=100,restore_best_weights=True)

# Checkpoint to save model

chkpt = ModelCheckpoint(filepath = 'model_checkpoint.h5', save_best_only=True)

# number of training and validation steps for training and validation
nb_train_steps = int(np.ceil(len(train_df)/batch_size))
nb_valid_steps = int(np.ceil(len(valid_df)/batch_size))

# number of epochs
nb_epochs = 50

In [ ]:
# train the model
history = model.fit_generator(train_data_gen, epochs=nb_epochs,
                              steps_per_epoch=nb_train_steps, validation_data=valid_data_gen,
                              validation_steps=nb_valid_steps, callbacks=[es, chkpt])

Epoch 1/50


<ipython-input-23-85de4655e639>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_data_gen, epochs=nb_epochs,


531/532 [============================>.] - ETA: 0s - loss: 0.6634 - accuracy: 0.4885

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


532/532 [==============================] - 48s 72ms/step - loss: 0.6625 - accuracy: 0.4883 - val_loss: 0.8773 - val_accuracy: 0.3877
Epoch 2/50
532/532 [==============================] - 32s 61ms/step - loss: 0.6425 - accuracy: 0.5496 - val_loss: 1.0522 - val_accuracy: 0.5169
Epoch 3/50
532/532 [==============================] - 31s 58ms/step - loss: 0.6256 - accuracy: 0.5557 - val_loss: 1.0232 - val_accuracy: 0.3919
Epoch 4/50
532/532 [==============================] - 33s 62ms/step - loss: 0.7434 - accuracy: 0.5414 - val_loss: 0.5444 - val_accuracy: 0.8559
Epoch 5/50
532/532 [==============================] - 32s 61ms/step - loss: 0.7744 - accuracy: 0.5289 - val_loss: 1.1996 - val_accuracy: 0.4025
Epoch 6/50
532/532 [==============================] - 30s 57ms/step - loss: 0.9029 - accuracy: 0.5202 - val_loss: 0.7376 - val_accuracy: 0.5381
Epoch 7/50
532/532 [==============================] - 30s 57ms/step - loss: 0.8443 - accuracy: 0.5322 - val_loss: 0.9208 - val_accuracy: 0.4979
Epo